In [ ]:
# !pip install datasets

import pickle
import torch
import numpy as np
import pandas as pd
import datasets
from datasets import Dataset
from datasets import load_metric
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification
from transformers import Trainer
from transformers import TrainingArguments
from transformers import BertConfig
from transformers import BertTokenizer
from transformers import AutoTokenizer
from transformers import BertForMaskedLM
from transformers import AutoModel
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from tqdm.notebook import trange, tqdm

In [ ]:
all_dataset = datasets.load_from_disk("data_class")

In [ ]:
all_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1138
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 356
    })
    eval: Dataset({
        features: ['text', 'label'],
        num_rows: 285
    })
})

In [ ]:
all_dataset['train']['text']

['you need to find some self love and it will do the trick',
 'As long as you are here and breathing, youre doing enough',
 'sing the ABCs in your head',
 'good on you for doing the right thing.',
 'I remember you telling me',
 'dad was extremely abusive',
 'I get hating yourself',
 "he doesn't deserve you",
 'solution is not always that simple.',
 'purpose beyond justthe sickness',
 'U self medicating on crack?',
 'works wonders for treatment-resistant depression',
 "I don't think most people would feel guilty",
 'it all depends on *you* and your individual',
 'Might help to',
 'I struggled with it',
 'I can hear the pain and distress in your voice',
 "you can't find safety",
 'You are never alone',
 'Yeah, you were just telling me',
 'Youre being treated, ',
 'Perhaps is not my fault',
 'what makes life worth living for you',
 'Do you have a group you can talk to,',
 'youre just hurting right now',
 'Lower your intake',
 'involved with my church',
 'I was terrible at math',
 'I somet

In [ ]:
all_dataset['train']['label']

[0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
# def preprocess_function(examples):
#     return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)
# def preprocess_function(examples):
#     return tokenizer(examples["text"])
tokenized_dataset = all_dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english",num_labels=2)

model.resize_token_embeddings(len(tokenizer))
# small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))

Loading cached processed dataset at data_class/train/cache-59db4c30084ed7b8.arrow
Loading cached processed dataset at data_class/test/cache-fb0e8f7c71adce66.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Embedding(30522, 768, padding_idx=0)

In [ ]:
small_train_dataset = tokenized_dataset["train"]
small_eval_dataset = tokenized_dataset["eval"]
small_test_dataset = tokenized_dataset["test"]

In [ ]:
small_test_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 356
})

In [ ]:
small_eval_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 285
})

In [ ]:
small_train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1138
})

In [ ]:
training_args = TrainingArguments(
    output_dir="results_1",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1138
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 360


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=360, training_loss=0.34681578742133246, metrics={'train_runtime': 43559.0507, 'train_samples_per_second': 0.131, 'train_steps_per_second': 0.008, 'total_flos': 753739498352640.0, 'train_loss': 0.34681578742133246, 'epoch': 5.0})

In [ ]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 356
  Batch size = 16


{'eval_loss': 0.6550705432891846,
 'eval_accuracy': 0.7696629213483146,
 'eval_runtime': 219.5793,
 'eval_samples_per_second': 1.621,
 'eval_steps_per_second': 0.105}